In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect

In [16]:
name_basics = pd.read_csv('imdb.name.basics.csv.gz')
name_basics.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [2]:
title_principals = pd.read_csv('imdb.title.principals.csv.gz')
title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [3]:
title_pricipals_1 = title_principals.drop('job', axis=1)
title_pricipals_1.head()

,tconst,ordering,nconst,category,characters
0,tt0111414,1,nm0246005,actor,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN
2,tt0111414,3,nm3739909,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN
4,tt0323808,1,nm3579312,actress,"[""Beth Boothby""]"


In [17]:
main_pricipals = title_pricipals_1.drop('characters', axis=1)

In [18]:
title_basics = pd.read_csv('imdb.title.basics.csv.gz')
title_basics.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [6]:
title_basics_1 = title_basics.drop('original_title', axis=1)

In [7]:
title_basics_2 = title_basics_1.drop('start_year', axis=1)

In [8]:
main_titles = title_basics_2.drop('runtime_minutes', axis=1)

In [9]:
director = main_titles.merge(main_pricipals, on="tconst", how='inner')

In [10]:
director.head(30)

,tconst,primary_title,genres,ordering,nconst,category
0,tt0063540,Sunghursh,"Action,Crime,Drama",10,nm0006210,composer
1,tt0063540,Sunghursh,"Action,Crime,Drama",1,nm0474801,actor
2,tt0063540,Sunghursh,"Action,Crime,Drama",2,nm0904537,actress
3,tt0063540,Sunghursh,"Action,Crime,Drama",3,nm0756379,actor
4,tt0063540,Sunghursh,"Action,Crime,Drama",4,nm0474876,actor
5,tt0063540,Sunghursh,"Action,Crime,Drama",5,nm0712540,director
6,tt0063540,Sunghursh,"Action,Crime,Drama",6,nm0023551,writer
7,tt0063540,Sunghursh,"Action,Crime,Drama",7,nm1194313,writer
8,tt0063540,Sunghursh,"Action,Crime,Drama",8,nm0347899,writer
9,tt0063540,Sunghursh,"Action,Crime,Drama",9,nm1391276,writer


In [31]:
categories.drop_duplicates(subset ='primary_title',
                     keep = False, inplace = True)

<ipython-input-31-0a0cf95065e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories.drop_duplicates(subset ='primary_title',


In [73]:
categories = director[director['category'] == 'director']
# director.drop(categories, inplace=False)

In [46]:
categories["genres"] = categories["genres"].apply(lambda x: x.split(",") if type(x) == str else x)
categories

<ipython-input-46-43baf5de3fc0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories["genres"] = categories["genres"].apply(lambda x: x.split(",") if type(x) == str else x)


,tconst,primary_title,genres,ordering,nconst,category
5,tt0063540,Sunghursh,"[Action, Crime, Drama]",5,nm0712540,director
14,tt0066787,One Day Before the Rainy Season,"[Biography, Drama]",5,nm0002411,director
22,tt0069049,The Other Side of the Wind,[Drama],5,nm0000080,director
32,tt0069204,Sabse Bada Sukh,"[Comedy, Drama]",5,nm0611531,director
42,tt0100275,The Wandering Soap Opera,"[Comedy, Drama, Fantasy]",5,nm0749914,director
...,...,...,...,...,...,...
1028153,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,[Documentary],6,nm9272491,director
1028162,tt9916706,Dankyavar Danka,[Comedy],5,nm7764440,director
1028171,tt9916730,6 Gunn,NaN,5,nm10538612,director
1028181,tt9916754,Chico Albuquerque - Revelações,[Documentary],5,nm9272490,director


In [72]:
categories_1 = (categories.explode('genres'))

In [71]:
genres = categories_1[categories_1['genres'] == 'Adventure']
# director.drop(genres, inplace=False)

In [61]:
name_basics_1 = name_basics.drop('birth_year', axis=1)

In [63]:
name_basics_2 = name_basics_1.drop('death_year', axis=1)

,nconst,primary_name,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [64]:
name_basics_3 = name_basics_2.drop('known_for_titles', axis=1)

In [70]:
name_basics_3["primary_profession"] = name_basics_3["primary_profession"].apply(lambda x: x.split(",") if type(x) == str else x)

In [69]:
name_basics_4 = (name_basics_3.explode('primary_profession'))

In [80]:
final_name = name_basics_4[name_basics_4['primary_profession'] == 'director']
# director.drop(primary_profession, inplace=False)

In [79]:
director_merge = genres.merge(final_name, on="nconst", how='inner')
director_merge.head()

,tconst,primary_title,genres,ordering,nconst,category,primary_name,primary_profession
0,tt0293429,Mortal Kombat,Adventure,1,nm2585406,director,Simon McQuoid,director
1,tt0312305,Quantum Quest: A Cassini Space Odyssey,Adventure,6,nm0820800,director,Daniel St. Pierre,director
2,tt0884726,Legends of Oz: Dorothy's Return,Adventure,6,nm0820800,director,Daniel St. Pierre,director
3,tt0329539,Rice Girl,Adventure,5,nm0278780,director,Michael Fischa,director
4,tt0331314,Bunyan and Babe,Adventure,5,nm8625898,director,Louis Ross,director
